In [0]:
dbutils.fs.unmount('/mnt/files/')

In [0]:
storageAccountName = 'retailanalyticssa'
blobContainerName = 'prac'
storageAccountAccessKey = dbutils.secrets.get(scope = "sa_key", key = "sakey")

dbutils.fs.mount(
  source = f'wasbs://{blobContainerName}@{storageAccountName}.blob.core.windows.net',
  mount_point = '/mnt/files/',
  extra_configs = {'fs.azure.account.key.' + storageAccountName + '.blob.core.windows.net': storageAccountAccessKey}
)

In [0]:
%fs 
ls /mnt/files/data_json_orders

In [0]:
from pyspark.sql.functions import *

In [0]:
orders_schema = "order_id long,customer_id long,customer_fname string,customer_lname string,city string,state string,pincode long,line_items array<struct<order_item_id: long,order_item_product_id: long,order_item_quantity: long,order_item_product_price: float,order_item_subtotal: float>>"

In [0]:
orders_df = spark \
.readStream \
.format("json") \
.schema(orders_schema) \
.option("path","/FileStore/tables/input") \
.load()

In [0]:
df1 = spark \
.read \
.format("json") \
.schema(orders_schema) \
.option("path","/FileStore/tables/input") \
.load()

In [0]:
df1.show()

In [0]:
orders_df.createOrReplaceTempView("orders")

In [0]:
exploded_orders = spark.sql("""select order_id, customer_id, city, state,
                                pincode, explode(line_items) lines 
                                from orders""")

In [0]:
exploded_orders.createOrReplaceTempView("exploded_orders")

In [0]:
flattened_orders = spark.sql("""select order_id, customer_id, city, state, pincode, 
                                lines.order_item_id as item_id, 
                                lines.order_item_product_id as product_id,
                                lines.order_item_quantity as quantity, 
                                lines.order_item_product_price as price,
                                lines.order_item_subtotal as subtotal 
                                from exploded_orders""")

In [0]:
flattened_orders.createOrReplaceTempView("orders_flattened")

In [0]:
aggregated_orders = spark.sql("""select customer_id, approx_count_distinct(order_id) as orders_placed, 
                                count(item_id) as products_purchased, sum(subtotal) as amount_spent 
                                from orders_flattened 
                                group by customer_id""")

In [0]:
%fs 
rm -r dbfs:/FileStore/tables/checkpointdir1

In [0]:
%fs 
rm -r /FileStore/tables/input/file1.json

In [0]:

%fs 
rm -r dbfs:/user/hive/warehouse/stream1.db/orders_result1

In [0]:
%fs 
rm -r dbfs:/user/hive/warehouse/stream1.db

In [0]:
spark.sql("drop table stream1.orders_result1")
spark.sql("drop database stream1")

In [0]:
spark.sql("create database stream1")

In [0]:
streaming_query = aggregated_orders \
.writeStream \
.format("delta") \
.outputMode("complete") \
.option("checkpointLocation","/FileStore/tables/checkpointdir1") \
.toTable("orders_result1")

In [0]:
spark.sql("show tables").show()

In [0]:
#spark.sql("use database stream1")
spark.sql("select * from orders_result1").show()

In [0]:
streaming_query.explain()

In [0]:
streaming_query.stop()